In [6]:
# Imports
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from tetris_main import *
import random
from library.hill_climbing.hc import hill_climb
from library.simulated_annealing.sa import simulated_annealing
from library.genetic_algorithm import gao

In [7]:
grid = np.zeros((4,5))
T_DICT = {
    "I": [(0, 0), (1, 0), (2, 0), (3, 0)],
    "L": [(0, 0), (1, 0), (2, 0), (2, 1)],
    "J": [(0, 0), (1, 0), (2, 0), (2, -1)],
    "T": [(0, 0), (1, 0), (1, -1), (1, 1)],
    "S": [(0, 0), (1, 0), (1, -1), (0, 1)],
    "Z": [(0, 0), (1, 0), (1, 1), (0, -1)],
    "O": [(0, 0), (1, 0), (0, 1), (1, 1)],
}

pieces = ["L4","J3","Z2","I1","I1"]
grid, pieces_coordinates = tetrimino_fitter(pieces, (5,4))

In [8]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        yaxis=dict(range=[-0.5, grid.shape[1] - 0.5], fixedrange=True,),
        xaxis=dict(range=[-0.5, grid.shape[0] - 0.5], tickfont_size=12,),
    ),
)
fig.write_html(f"graphs/test_{grid.size}_size.html")
fig.show()


In [9]:
pop_size = 8
grid_shape = (5,4)
pieces_list = ["L","J","Z","I","I"]
individuals = [generate_individual(pieces_list) for i in range(pop_size)]
pop = Population(individuals, "max", 5, pieces_list, grid_shape)

In [14]:
pop.elites

[Individual <2000.0>, Individual <2000.0>]

In [13]:
pop = Population(individuals, "max", 2, pieces_list, grid_shape)
gao.GAO(
    pop,
    5,
    tournament_size=4,
    p_crossover=0.05,
    p_mutation=0.1,
    n_mutations=10,
    p_rot_mut=0.1,
    p_adoption=0.05,
)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 1],
            y=pieces_coordinates[i][:, 0],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=500,
        height=500,
        title=f"Fitness: {int(pop.elites[0].fitness)}/{int(np.sum(100*np.ones_like(grid)))} // Pieces used: {len(pieces_coordinates)}/{len(pieces_list)}",
        xaxis=dict(range=[-0.5, grid.shape[1] - 0.5], fixedrange=True,),
        yaxis=dict(range=[-0.5, grid.shape[0] - 0.5], tickfont_size=12,),
    ),
)
fig.show()
